# Log Metrics Over Time

In this notebook we showcase how to track and visualize the temporal evolution of key model performance metrics, such as AUC, F1 score, precision, recall, and accuracy. These tests are useful for analyzing the stability and trends in model performance indicators, helping to identify potential degradation or unexpected fluctuations in model behavior over time. By monitoring these metrics systematically, teams can detect early warning signs of model drift and take proactive measures to maintain model reliability.

ValidMind allows you to log metrics over time using the `log_metric()` function from the `validmind-library` and visualize them in your documentation using the *Metrics Over Time* block. This integration enables seamless tracking of model performance, with support for custom thresholds and automated alerting capabilities.

<a id='toc1_'></a>

## About ValidMind
ValidMind is a suite of tools for managing model risk, including risk associated with AI and statistical models.

You use the ValidMind Library to automate documentation and validation tests, and then use the ValidMind Platform to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin
This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?
If you haven't already seen our [Get started with the ValidMind Library](https://docs.validmind.ai/developer/get-started-validmind-library.html), we recommend you begin by exploring the available resources in this section. There, you can learn more about documenting models, find code samples, or read our developer reference.

<div class="alert alert-block alert-info" style="background-color: #B5B5B510; color: black; border: 1px solid #083E44; border-left-width: 5px; box-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2);border-radius: 5px;"><span style="color: #083E44;"><b>For access to all features available in this notebook, create a free ValidMind account.</b></span>
<br></br>
Signing up is FREE — <a href="https://docs.validmind.ai/guide/configuration/register-with-validmind.html" style="color: #DE257E;"><b>Register with ValidMind</b></a></div>

<a id='toc1_3_'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Library, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Custom tests**: Custom tests are functions that you define to evaluate your model or dataset. These functions can be registered via the ValidMind Library to be used with the ValidMind Platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind Library. They can be any of the following:

- **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
- **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
- **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom test.
- **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom test. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a test, customize its behavior, or provide additional context.

**Outputs**: Custom tests can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: The [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.

<a id='toc2_'></a>

## Install the ValidMind Library

To install the library:

In [ ]:
%pip install -q validmind

<a id='toc3_'></a>

## Initialize the ValidMind Library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the ValidMind Library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

<a id='toc3_1_'></a>

### Get your code snippet

1. In a browser, [log in to ValidMind](https://docs.validmind.ai/guide/configuration/log-in-to-validmind.html).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register Model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Credit Risk Scorecard`
   - Use case: `Credit Risk - CECL`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace the placeholder with your own code snippet:

In [ ]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  model = "...",
  monitoring = True
)

<a id='toc4_'></a>

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [3]:
import xgboost as xgb
import numpy as np

from datetime import datetime, timedelta

from validmind.unit_metrics import list_metrics, describe_metric, run_metric
from validmind.api_client import log_metric

%matplotlib inline

## Load demo model

We use a classification model trained on customer churn data to demonstrate ValidMind's metric logging capabilities. The notebook uses a built-in classification dataset, processes it through train-validation-test splits, and trains an XGBoost classifier. The trained model and datasets are then initialized in ValidMind's framework, enabling us to track and monitor various performance metrics in the following sections.

In [ ]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{customer_churn.target_column}' \n\t• Class labels: {customer_churn.class_labels}"
)

raw_df = customer_churn.load_data()
raw_df.head()

In [ ]:
train_df, validation_df, test_df = customer_churn.preprocess(raw_df)

x_train = train_df.drop(customer_churn.target_column, axis=1)
y_train = train_df[customer_churn.target_column]
x_val = validation_df.drop(customer_churn.target_column, axis=1)
y_val = validation_df[customer_churn.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

Once the datasets and model are prepared for validation, we initialize ValidMind `dataset` and `model`, specifying features and targets columns. The property `input_id` allows users to uniquely identify each dataset and model. This allows for the creation of multiple versions of datasets and models, enabling us to compute metrics by specifying which versions we want to use as inputs.

In [6]:
vm_raw_dataset = vm.init_dataset(
    dataset=raw_df,
    input_id="raw_dataset",
    target_column=customer_churn.target_column,
    class_labels=customer_churn.class_labels,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=customer_churn.target_column,
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, input_id="test_dataset", target_column=customer_churn.target_column
)

vm_model = vm.init_model(
    model,
    input_id="model",
)

We can now use the `assign_predictions()` method from the Dataset object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
)

vm_test_ds.assign_predictions(
    model=vm_model,
)

## Log metrics

In this section, we demonstrate how to track the temporal evolution of key model performance metrics, such as AUC, F1 score, precision, recall, and accuracy. These tests are useful for analyzing the stability and trends in model performance indicators, helping to identify potential degradation or unexpected fluctuations in model behavior over time. Unit metrics in ValidMind provide a standardized way to compute and track individual performance measures, making it easy to monitor specific aspects of model behavior. 

We'll showcase how to set appropriate thresholds for each metric, enabling automated alerting when performance drifts beyond acceptable boundaries, and demonstrate how these thresholds can be customized based on business requirements and risk tolerance levels.

### List Available Metrics 

We can list all available unit metrics by using the `list_metrics()` function from the `unit_metrics` module. This function returns a list of all available metric functions.

In [ ]:
metrics = [metric for metric in list_metrics() if "classification" in metric]

for metric_id in metrics:
    describe_metric(metric_id)

### Run unit metrics

To compute individual metrics, we will use ValidMind's unit metrics - single-value metrics that can be computed on a dataset and model. Using the `run_metric()` function from the `validmind.unit_metrics` module, we can calculate these metrics. While this function has a signature similar to `run_test()` from the `validmind.tests` module, it is specifically designed for unit metrics and takes the following arguments:

- `metric_id`: The unique identifier for the metric (e.g., `validmind.unit_metrics.classification.ROC_AUC`)
- `inputs`: A dictionary containing the input dataset and model or their respective input IDs
- `params`: A dictionary containing keyword arguments for the unit metric (optional, accepts any `kwargs` from the underlying sklearn implementation)

The function returns and displays a result object similar to a normal ValidMind test, but only shows the unit metric value. While this result object has a `.log()` method for logging to the ValidMind Platform, in this use case we'll use unit metrics to compute performance metrics and then log them over time using the `log_metric()` function from the `validmind.api_client` module.

In [ ]:
result = run_metric(
    "validmind.unit_metrics.classification.ROC_AUC",
    inputs={
        "model": vm_model,
        "dataset": vm_test_ds,
    },
)
auc = result.metric

In [ ]:
result = run_metric(
    "validmind.unit_metrics.classification.Accuracy",
    inputs={
        "model": vm_model,
        "dataset": vm_test_ds,
    },
)
accuracy = result.metric

In [ ]:
result = run_metric(
    "validmind.unit_metrics.classification.Recall",
    inputs={
        "model": vm_model,
        "dataset": vm_test_ds,
    },
)
recall = result.metric

In [ ]:
f1 = run_metric(
    "validmind.unit_metrics.classification.F1",
    inputs={
        "model": vm_model,
        "dataset": vm_test_ds,
    },
)
f1 = result.metric

In [ ]:
precision = run_metric(
    "validmind.unit_metrics.classification.Precision",
    inputs={
        "model": vm_model,
        "dataset": vm_test_ds,
    },
)
precision = result.metric

### Log unit metrics over time

Using the `log_metric()` function from the `validmind.api_client` module, we can log the unit metrics over time. This function takes the following arguments:

- `key`: The name of the metric to log
- `value`: The value of the metric to log
- `recorded_at`: The timestamp of the metric to log
- `thresholds`: A dictionary containing the thresholds for the metric to log
- `params`: A dictionary containing the keyword arguments for the unit metric (in this case, none are required, but we can pass any `kwargs` that the underlying sklearn implementation accepts)

In [14]:
log_metric(
    key="AUC Score",
    value=auc,
    recorded_at=datetime(2024, 1, 1),
)

To visualize the logged metric, we can use the *Metrics Over Time* block in ValidMind. After adding this visualization block to your documentation (as shown in the image below), you'll be able to see your logged metrics plotted over time. In this example, since we've only logged a single data point, the visualization shows just one measurement. As you continue logging metrics, the graph will populate with more points, enabling you to track trends and patterns.

![Metric Over Time block](../images/add_metric_over_time_block.png)
![AUC Score](../images/log_metric_auc_1.png)

### Pass thresholds

We can pass thresholds to the `log_metric()` function to visualize the metric over time. This is useful for visualizing the metric over time and identifying potential issues. The metric visualization component provides a dynamic way to monitor and contextualize metric values through customizable thresholds. These thresholds appear as horizontal reference lines on the chart. The system always displays the most recent threshold configuration, meaning that if you update threshold values in your client application, the visualization will reflect these changes immediately. 

When a metric is logged without thresholds or with an empty threshold dictionary, the reference lines gracefully disappear from the chart, though the metric line itself remains visible. Thresholds are highly flexible in their implementation. You can define them with any meaningful key names (such as "low_risk", "maximum", "target", or "acceptable_range") in your metric data, and the visualization will adapt accordingly. 

In [15]:
log_metric(
    key="AUC Score",
    value=auc,
    recorded_at=datetime(2024, 1, 1),
    thresholds={
        "min_auc": 0.7,
    }
)

![AUC Score](../images/log_metric_auc_2.png)

In [16]:
log_metric(
    key="AUC Score",
    value=auc,
    recorded_at=datetime(2024, 1, 1),
    thresholds={
        "high_risk": 0.6,
        "medium_risk": 0.7,
        "low_risk": 0.8,
    }
)

![AUC Score](../images/log_metric_auc_3.png)

### Log multiple metrics with custom thresholds

The following code snippet shows an example of how to set up and log multiple performance metrics with custom thresholds for each metric. Using AUC, F1, Precision, Recall, and Accuracy scores as examples, it demonstrates how to define different risk levels (high, medium, low) appropriate for each metric's expected range. The code simulates 10 days of metric history by applying a gradual decay and random noise to help visualize how metrics might drift over time in a production environment.

In [ ]:
NUM_DAYS = 10
REFERENCE_DATE = datetime(2024, 1, 1)  # Fixed date: January 1st, 2024
base_date = REFERENCE_DATE - timedelta(days=NUM_DAYS)

# Initial values with their specific thresholds
performance_metrics = {
    "AUC Score": {
        "value": auc,
        "thresholds": {
            "high_risk": 0.7,
            "medium_risk": 0.8,
            "low_risk": 0.9,
        }
    },
    "F1 Score": {
        "value": f1,
        "thresholds": {
            "high_risk": 0.5,
            "medium_risk": 0.6,
            "low_risk": 0.7,
        }
    },
    "Precision Score": {
        "value": precision,
        "thresholds": {
            "high_risk": 0.6,
            "medium_risk": 0.7,
            "low_risk": 0.8,
        }
    },
    "Recall Score": {
        "value": recall,
        "thresholds": {
            "high_risk": 0.4,
            "medium_risk": 0.5,
            "low_risk": 0.6,
        }
    },
    "Accuracy Score": {
        "value": accuracy,
        "thresholds": {
            "high_risk": 0.75,
            "medium_risk": 0.8,
            "low_risk": 0.85,
        }
    }
}

# Trend parameters
trend_factor = 0.98  # Slight downward trend
noise_scale = 0.02   # Random fluctuation of ±2%

for i in range(NUM_DAYS):
    recorded_at = base_date + timedelta(days=i)
    print(f"\nrecorded_at: {recorded_at}")

    # Log each metric with trend and noise
    for metric_name, metric_info in performance_metrics.items():
        base_value = metric_info["value"]
        thresholds = metric_info["thresholds"]
        
        # Apply trend and add random noise
        trend = base_value * (trend_factor ** i)
        noise = np.random.normal(0, noise_scale * base_value)
        value = max(0, min(1, trend + noise))  # Ensure value stays between 0 and 1
        
        log_metric(
            key=metric_name,
            value=value,
            recorded_at=recorded_at.isoformat(),
            thresholds=thresholds
        )
        
        print(f"{metric_name:<15}: {value:.4f} (Thresholds: {thresholds})")

![AUC Score](../images/log_metric_auc_4.png)
![Accuracy Score](../images/log_metric_accuracy.png)
![Precision Score](../images/log_metric_precision.png)
![Recall Score](../images/log_metric_recall.png)
![F1 Score](../images/log_metric_f1.png)